1. Importing and converting to Python Object

In [7]:
import json
import re
from functools import reduce
import glob

Load data from fsm_data folder. Currently, the manual analysis do have the game DidYouCatchTheBall, TheDice, and Tangram so we temporary remove them until we receive manual analysis.


In [8]:
main_dir = '../fsm-data/'
f1 = open(main_dir + 'DidYouCatchTheBall_US.json')
f2 = open(main_dir + 'fireballs20-US.json')
f3 = open(main_dir + 'Hackers_IN.json')
f4 = open(main_dir + 'Indianboys_IN.json')
f5 = open(main_dir + 'Indiangirls_IN.json')
f6 = open(main_dir + 'MathParkour_US.json')
f7 = open(main_dir + 'Raistar_IN.json')
f8 = open(main_dir + 'TangramsRace.json')
f9 = open(main_dir + 'TheDice_US.json')
data1 = json.load(f1)
data2 = json.load(f2)
data3 = json.load(f3)
data4 = json.load(f4)
data5 = json.load(f5)
data6 = json.load(f6)
data7 = json.load(f7)
data8 = json.load(f8)
data9 = json.load(f9)

In [9]:
data = []
dir = "../fsm-data/*.json"
old_files = sorted(glob.glob(dir, recursive=False))
files = []
# Until we got enough data, we can comment out from

#here
for file in old_files:
  if(file != "../fsm-data\TheDice_US.json" and file != "../fsm-data\DidYouCatchTheBall_US.json" and file != "../fsm-data\TangramsRace.json"):
    files.append(file)
#to_here

print(len(files))
for single_file in files:
    with open(single_file, 'r') as f:
        json_file = json.load(f)
        # new_string = json.dumps(json_file, indent = 2)
        data.append(json_file)

6


Testing purpose

In [10]:
new_string = json.dumps(data, indent = 2)
new_string1 = json.dumps(data, indent = 2)
# print(new_string)
# print(new_string1)

2. Defining Graph

In [11]:
class Graph:
    global adj, cnt
    def __init__(self, V): #this part like a constructor...
        self.V = V
        self.adj = [[] for i in range(V)]
        self.cnt = 0

    def add_edge(self, source, dest):
        self.adj[source].append(dest)
        
    def is_cyclic_util(self, i, visited, rec_stack):
        if(rec_stack[i]):
            self.cnt += 1
            return
        if(visited[i]): return
        visited[i] = True
        rec_stack[i] = True
        children = self.adj[i]
        
        for c in children:
            self.is_cyclic_util(c, visited, rec_stack)
        
        rec_stack[i] = False
        return
    
    def count(self):
        visited = [False for i in range(self.V)]
        rec_stack = [False for i in range(self.V)]
        for i in range(self.V):
            self.is_cyclic_util(i, visited, rec_stack)
        return self.cnt

3. Methods for counting loops through states and connections

In [12]:
#this method hashing state to its corresponding index
#Important: if the number of state does not
def index(s, dic, idx):
    tmp = re.findall("\d+\Z", s)
    return 0 if len(tmp) == 0 else int(tmp[0])
# print(index("74122bc887b34237fd75ac304b9c55bd_state_10"))

In [13]:
#this method create a dictonary: connection = (state_from, state_to)
#return type: dictionary
def con_to_state(lst):
    dic = {}
    idx = 0
    state_to_idx = {}
    for e in lst:
        if(state_to_idx.get(e["connectionFrom"]) == None):
            state_to_idx[e["connectionFrom"]] = idx
            idx += 1
        if(state_to_idx.get(e["connectionTo"]) == None):
            state_to_idx[e["connectionTo"]] = idx
            idx += 1
        dic[e["connectionId"]] = (state_to_idx[e["connectionFrom"]], state_to_idx[e["connectionTo"]])
    return dic


In [14]:
def create_graph(data):
    graph = Graph(len(data["states"]))
        
    conv = con_to_state(data["connections"])
    
    for e in data["states"]:
        # print(e["outputConnections"])
        for oc in e["outputConnections"]:
            start = conv[oc][0]
            end = conv[oc][1]
            graph.add_edge(start, end)
    # print(graph.adj)
    return graph


In [15]:
def count_loop(graph):
    return graph.count()

def count_if_else(graph):
    return reduce(lambda acc, cur: acc + 1 if len(cur) > 1 else acc, graph.adj, 0)

def traversal(array):
    res = []
    for idx, e in enumerate(array):
        graph = create_graph(e)
        loop = count_loop(graph)
        if_else = count_if_else(graph)
        print("There are " + str(loop) + " loops and there are " + str(if_else) + " branches in the " + str(idx + 1) + " graph\n")
        res.append((loop, if_else))
    return #res
# you can return loop and if_else as an array of tuples by comment out the #res

4. Implementing: counting loops and count if-else statements

In [16]:
traversal(data)

There are 2 loops and there are 2 branches in the 1 graph

There are 0 loops and there are 0 branches in the 2 graph

There are 1 loops and there are 1 branches in the 3 graph

There are 0 loops and there are 0 branches in the 4 graph

There are 4 loops and there are 4 branches in the 5 graph

There are 1 loops and there are 2 branches in the 6 graph

